Ноутбук необходим для добавления новых адресов с геокодом в общую базу адресов.

Также по каждому коду нужно отдельно выделить дополнительные колонки как долгота и широта, уникальный номер региона и гексагон h3

In [2]:
from geojson import Point, Polygon, Feature
import pandas as pd
import h3
import json
from turfpy.measurement import boolean_point_in_polygon

#### Добавление новых данных в общую базу геокодинга

In [ ]:
def read_and_add(base,adding_file):
    base_adding_file = pd.concat([base,adding_file])
    base_adding_file.reset_index(drop=True,inplace=True)
    base_adding_file.to_csv('../data/processed_data/kaz_geocash.csv')
    
    base_adding_file_notna = base_adding_file.dropna()
    base_adding_file_notna.to_csv('../data/processed_data/kaz_geocash_notna.csv')
    return base_adding_file_notna

In [3]:
kz_total = pd.read_csv('../data/processed_data/kaz_geocash.csv',index_col=0)
add = pd.read_csv('../data/interim_data/region_data/res_1.csv',index_col=0)

#### Добавление долготы и широты

In [51]:
def add_lat_lon(new_geodata):
    new_geodata[['lon', 'lat']] = new_geodata['point'].str.split(' ',1,expand=True)
    new_geodata[['lon','lat']] = new_geodata[['lon','lat']].astype('float')
    return new_geodata

In [55]:
add_1 = add_lat_lon(add_1)

### Добавление гексагона 10 размера

In [23]:
def create_hexagon_clusters(data, low_level_res = 12):
    """low_level_res & high_level_res размеры гексагонов нижнего и верхнего уровня"""
    #создание кластера нижнего уровня
    data['cluster'] = [h3.geo_to_h3(row['lat'], row['lon'], low_level_res) for idx, row in data.iterrows()]
    return data

In [56]:
add_1 = create_hexagon_clusters(add_1)

### Добавить код района

In [ ]:
with open('../data/raw_data/kz_county_polygons.json','r') as f:
    data = json.loads(f.read())

In [58]:
def add_county_code(new_geodata, dict_region_code):
    new_geodata['coords'] = list(zip(new_geodata['lon'],new_geodata['lat']))
    new_geodata['coords'] = new_geodata['coords'].apply(Point)
    
    dict_region_code = pd.json_normalize(dict_region_code, record_path =['features'])
    dict_region_code['geometry.coordinates'] = dict_region_code['geometry.coordinates'].apply(Polygon)
    
    new_geodata.reset_index(drop=True,inplace=True)
    
    new_geodata['region_code'] = 0
    for i in range(len(new_geodata)):
        for j in range(len(dict_region_code)):
            if boolean_point_in_polygon(new_geodata['coords'][i], dict_region_code['geometry.coordinates'][j]):
                new_geodata['region_code'][i] = dict_region_code['properties.GID_2'][j]
            else:
                continue
    return new_geodata

In [60]:
add_county_code(add_1[:100], data)

<ipython-input-58-1a99e6fbdb04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_geodata['coords'] = list(zip(new_geodata['lon'],new_geodata['lat']))
<ipython-input-58-1a99e6fbdb04>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_geodata['coords'] = new_geodata['coords'].apply(Point)
<ipython-input-58-1a99e6fbdb04>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,bin,full_add,point,precision,lon,lat,cluster,coords,region_code
0,590918401809,"обл. Восточно-Казахстанская,г. Семей,ВАЛИХАНОВ...",80.261291 50.410964,exact,80.261291,50.410964,8c206263540e9ff,"{'type': 'Point', 'coordinates': [80.261291, 5...",KAZ.5.10_1
1,190940004745,"обл. Туркестанская,пос. Асыката,АМАНГЕЛДИ,33",68.367129 40.895671,other,68.367129,40.895671,8c20a54f4d4ebff,"{'type': 'Point', 'coordinates': [68.367129, 4...",KAZ.12.5_1
2,191140006933,"обл. Костанайская,г. Костанай,Омара Дощанова,186а",63.604576 53.222002,exact,63.604576,53.222002,8c21294e0b705ff,"{'type': 'Point', 'coordinates': [63.604576, 5...",KAZ.10.13_1
3,580217401589,"обл. Павлодарская,с. Теренколь,ул.Тургенева,84",76.099953 53.068141,street,76.099953,53.068141,8c215d308b521ff,"{'type': 'Point', 'coordinates': [76.099953, 5...",KAZ.8.6_1
4,890722400372,"обл. Атырауская,пос. Индерборский,ул.Кунаева,13",NaN,none,NaN,NaN,0,"{'type': 'Point', 'coordinates': [nan, nan]}",0
...,...,...,...,...,...,...,...,...,...
95,780106401963,"обл. Туркестанская,с. Аксу,Жибек жолы,б/н",69.365481 41.477152,other,69.365481,41.477152,8c20a525b6923ff,"{'type': 'Point', 'coordinates': [69.365481, 4...",KAZ.12.8_1
96,820122400417,"обл. Актюбинская,р-он Астана,пр.А. Молдагулово...",57.17148 50.292695,exact,57.171480,50.292695,8c213196aa99dff,"{'type': 'Point', 'coordinates': [57.17148, 50...",KAZ.3.2_1
97,890340000083,"г. Алматы,р-он Алмалинский,ул. Амангельды,52",76.9312 43.254749,exact,76.931200,43.254749,8c20e60124d03ff,"{'type': 'Point', 'coordinates': [76.9312, 43....",KAZ.1.3_1
98,860305400131,"обл. Западно-Казахстанская,с. Сайхин,Т.Жароков...",46.76588 48.817863,street,46.765880,48.817863,8c1096c0c1767ff,"{'type': 'Point', 'coordinates': [46.76588, 48...",KAZ.13.11_1


In [50]:
kz_total_notna.to_csv('../data/processed_data/kaz_geocash_notna.csv')